In [13]:
# %load_ext autoreload
# %autoreload

In [14]:
## Adding 'FDS profiling package'
import sys
sys.path.insert(0, '/Users/harsh/Desktop/DATA_SCIENCE/PandasProfiling/')

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from missingno import missingno
from fds_profiling.visualisation.image_encoding import hex_to_rgb, plot_360_n0sc0pe

from fds_profiling.report import templates
from fds_profiling.report.renderable import Renderable

from fds_profiling.config import config

from fds_profiling.model.base import get_df_col_types
from fds_profiling.model.overview import overview

##### Titanic dataset

In [15]:
import requests
import io
s = requests.get("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv").content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [16]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [17]:
# df['Survived'].value_counts().plot.barh()
# plt.subplots_adjust(left=0.1, right=0.9, top=0.7, bottom=0.2)
# d = plot_360_n0sc0pe(plt)

##### Variable types

In [18]:
config.config["var_types"] = get_df_col_types(df)

##### Metrics

In [19]:
## TODO: space problem in metric name
metrics = {"Metric_Count": lambda x: x["Survived"].count()}

metrics["Metric_Sum"] = lambda x: x["Survived"].sum()
# user_inputs = ["sum", "avg"]

total_metrics = metrics

In [20]:
# #### Metric 1: Count
# df_1 = df.groupby('Name', as_index=False).agg({"Survived":'count'})
# df_1.rename(columns={"Survived":"count"}, inplace=True)

# #### Metric 2: Sum
# df_2 = df.groupby('Name', as_index=False).agg({"Survived":'sum'})
# df_2.rename(columns={"Survived":"sum"}, inplace=True)

# join_df = pd.merge(df_1, df_2, on=['Name'], how="outer")

In [21]:
# join_df

#### Report Structure

In [22]:
# data = [["Hello",10],['Bob',12],['Clarke',13]]
# df_4 = pd.DataFrame(data,columns=['Name','Age'])

# data = [["World",10],['Bob',12],['Clarke',13]]
# df_5 = pd.DataFrame(data,columns=['Name','Age'])

In [23]:
## name, anchor_id, render
sections = []
overview_df = overview(df)

## Overview
sections.append(
    Renderable(
        content = {
            "table_1_heading": "Dataset statistics",
            "table_1_content": overview_df,
            "table_2_heading": "Variables types",
            "table_2_content": overview_df
        },
        name="Overview",
        anchor_id="overview",
        type_id="table_table"))

## Variables
sections.append(
    Renderable(
        content = {"dataframe":df, "metrics":total_metrics, "column_types":config.config["var_types"]},
        name="Variables",
        anchor_id="variables",
        type_id="variables_container"))

##### Creating HTML report

In [24]:
## Creating
html_template = templates.template("report.html")
#### navigations items
nav_items = [
    (section.name, section.anchor_id)
    for section in sections
]

data = html_template.render(
    title = "Pandas Profiling", ## report
    nav_items=nav_items, ## navigation
    sections=sections, ## report (content)
)

## Writing
with open("sample_profiles/harsh_profiling.html", "w") as file:
    file.write(data)

In [2]:
# print (data)

##### Image checking

In [23]:
def missing_matrix(data: pd.DataFrame) -> str:
    """Generate missing values matrix plot
    Args:
      data: Pandas DataFrame to generate missing values matrix from.
    Returns:
      The resulting missing values matrix encoded as a string.
    """
    labels = True
    missingno.matrix(
        data,
        figsize=(10, 4),
        color=hex_to_rgb("#337ab7"),
        fontsize=10,
        sparkline=False,
        labels=labels,
    )
    plt.subplots_adjust(left=0.1, right=0.9, top=0.7, bottom=0.2)
    return plot_360_n0sc0pe(plt)

In [24]:
# html_template = templates.template("temp.html")

# data = html_template.render(
#     content = missing_matrix(df)
# )

# ## Writing
# with open("sample_profiles/temp_profiling.html", "w") as file:
#     file.write(data)

In [25]:
# print (missing_matrix(df))

In [29]:
# missingno.matrix(
#         df,
#         figsize=(10, 4),
#         color=hex_to_rgb("#337ab7"),
#         fontsize=10,
#         sparkline=False,
# #         labels=labels,
#     )

In [17]:
for key, value in descriptive.items():
    print (key)

Distinct values
Missing values


In [14]:
b = pd.DataFrame([["a", "b", "c"], ["c", "d", "e"]])

In [15]:
b

,0,1,2
0,a,b,c
1,c,d,e


In [1]:
#no